In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean as _mean, stddev as _stddev, col , collect_list
from IPython.display import display

import numpy
from numpy import allclose
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [2]:
import findspark
findspark.init()
findspark.find()



'C:\\Program Files\\Spark\\spark-3.5.1-bin-hadoop3'

In [3]:


# spark_home = "C:\Program Files\Spark\spark-3.5.1-bin-hadoop3"

# # %%
# os.environ["SPARK_HOME"] = spark_home

# # Add Spark bin and executors to PATH
# os.environ["PATH"] += os.pathsep + os.path.join(spark_home, "bin")
# os.environ["PATH"] += os.pathsep + os.path.join(spark_home, "sbin")

# # Add Spark Python libraries to PYTHONPATH
# os.environ["PYTHONPATH"] = os.path.join(spark_home, "python") + os.pathsep + os.environ.get("PYTHONPATH", "")
# os.environ["PYTHONPATH"] += os.pathsep + os.path.join(spark_home, "python", "lib")

# # Add PySpark to the system path
# os.environ["PATH"] += os.pathsep + os.path.join(spark_home, "python", "lib", "pyspark.zip")
# os.environ["PATH"] += os.pathsep + os.path.join(spark_home, "python", "lib", "py4j-0.10.9-src.zip")

# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
# os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'

In [4]:
# Create a SparkSession
spark = SparkSession.builder \
    .appName("My Spark App") \
    .getOrCreate()

# load data

In [76]:
cleaned_df = spark.read.csv("cleaned_songs.csv", header=True, inferSchema=True)
cleaned_df = cleaned_df.na.drop()


18454


# prepare data for classification

In [85]:
# remove lyrics

cleaned_df = cleaned_df.drop("lyrics")
cleaned_df = cleaned_df.drop("language")

features = [col_name for col_name in cleaned_df.columns if col_name != "genre"]

for feature in features:
    cleaned_df = cleaned_df.filter( col(feature).cast("float").isNotNull() | col(feature).cast("int").isNotNull() | col(feature).cast("double").isNotNull() )
print(cleaned_df.count())


18444


In [ ]:

# for feature in features:
#     stats = cleaned_df.select(
#         _mean(col(feature)).alias("mean"), _stddev(col(feature)).alias("stddev")
#     ).collect()
#     mean = stats[0]["mean"]
#     stddev = stats[0]["stddev"]

#     # Normalize each feature using z-scoring
#     cleaned_df = cleaned_df.withColumn(feature, (col(feature) - mean) / stddev)

#     # make all values lies between -1 and 1

#     min_value = cleaned_df.agg({feature: "min"}).collect()[0][0]
#     max_value = cleaned_df.agg({feature: "max"}).collect()[0][0]

#     value = max(abs(min_value), abs(max_value))

#     cleaned_df = cleaned_df.withColumn(feature, (col(feature) / value))


In [89]:


# Function to map each value to a tuple of (value, value^2, 1)
def z_score_map_function(row):
    value = row   
    return [(value, value**2, 1)]

# Function to reduce tuples of (value, value^2, 1) to (sum, sum of squares, count)
def z_score_reduce_function(acc, value):
    return (acc[0] + value[0], acc[1] + value[1], acc[2] + value[2])
    
# Function to get absolute maximum value 
def abs_max_map_function(x):
    return [(abs(x[0]))]

# Function to get absolute maximum value
def abs_max_reduce_function(acc, value):
    return max(acc, value)

In [90]:
from pyspark.sql.types import DoubleType
for index , feature in enumerate(features):
    # cast to double
    cleaned_df = cleaned_df.withColumn(feature, col(feature).cast(DoubleType()))
    # Map step
    mapped_rdd = cleaned_df.rdd.flatMap(lambda x: z_score_map_function(x[index+1]))
    
    # Reduce step
    reduced_result = mapped_rdd.reduce(lambda acc, value: z_score_reduce_function(acc, value))

    # Calculate standard deviation
    sum_value = reduced_result[0]
    sum_of_squares = reduced_result[1]
    count = reduced_result[2]

    mean = sum_value / count
    variance = (sum_of_squares - (sum_value**2 / count)) / count
    stddev = variance ** 0.5
    
    #perform z-score normalization
    cleaned_df = cleaned_df.withColumn(feature, (col(feature) - mean) / stddev)
    
    # make all values lies between -1 and 1
    
    # get absolute maximum value
    mapped_rdd = cleaned_df.select(feature).rdd.flatMap(lambda x: abs_max_map_function(x))
    reduced_result = mapped_rdd.reduce(lambda acc, value: abs_max_reduce_function(acc, value))
    
    max_value = reduced_result
    
    cleaned_df = cleaned_df.withColumn(feature, (col(feature) / max_value))

    print("Standard Deviation:", stddev)
    print("Mean:", mean)
    print ("Max Value:", max_value)
    

Standard Deviation: 0.15006284923115365
Mean: 0.6440616189546732
Max Value: 3.6355541811304777
Standard Deviation: 0.18053479742696843
Mean: 0.6924546898720455
Max Value: 3.743071693120036
Standard Deviation: 3.613112190200497
Mean: 5.343743222728259
Max Value: 1.5654805274557133
Standard Deviation: 2.9195676771850687
Mean: -6.766851116894382
Max Value: 9.42473404474583
Standard Deviation: 0.49348376406225425
Mean: 0.5804597701149425
Max Value: 1.1762489718744142
Standard Deviation: 0.10219321073441535
Mean: 0.10615844719149863
Max Value: 7.944182856905773
Standard Deviation: 0.21779406630267883
Mean: 0.1753124830611581
Max Value: 3.7498152764357324
Standard Deviation: 0.16823171974959092
Mean: 0.051182114648124014
Max Value: 5.562672049865623
Standard Deviation: 0.1537739161262623
Mean: 0.1896400000000001
Max Value: 5.243802202045147
Standard Deviation: 0.22868521153550592
Mean: 0.52061645575797
Max Value: 2.276519991224444
Standard Deviation: 27.579466098151435
Mean: 120.799484059856

In [91]:
#export to csv
cleaned_df.toPandas().to_csv("normalized_songs.csv", index=False , header=True)

In [7]:
# Index the string column 'genre' to a numerical column 'indexed_genre'
stringIndexer = StringIndexer(inputCol="genre", outputCol="indexed_genre")
si_model = stringIndexer.fit(cleaned_df)
td = si_model.transform(cleaned_df)

assembler = VectorAssembler(inputCols=features, outputCol="features" , handleInvalid="skip")
td = assembler.transform(td)


# split the data into train and test sets
td = td.select("indexed_genre" , "features")

td.printSchema()


root
 |-- indexed_genre: double (nullable = false)
 |-- features: vector (nullable = true)



In [ ]:

display(td.limit(10).toPandas())


In [8]:
# Split the data into training and testing sets (80% training, 20% testing)
train_data, test_data = td.randomSplit([0.8, 0.2], seed=42)

# Initialize the RandomForestClassifier
rf = RandomForestClassifier(numTrees=30, maxDepth=20, labelCol="indexed_genre", seed=42,
                            leafCol="leafId")

# Train the model on the training data
model = rf.fit(train_data)

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
genres = set(train_labels)
genres = list(genres)
def get_score_confusion_matrix(model, test_Data, test_labels):
    
    score_matrix = [[0 for i in range(len(genres))] for j in range(len(genres))]
    
    for i in range(len(test_labels)):
        pred = model.predict([test_Data[i]])
        score_matrix[genres.index(test_labels[i])][genres.index(pred[0])] += 1
    #normalize the confusion matrix
    score_matrix = [[score_matrix[i][j]/sum(score_matrix[i]) for j in range(len(genres))] for i in range(len(genres))]
    return score_matrix

def draw_confusion_matrix(confusion_matrix):
    plt.figure(figsize=(16, 5))
    sns.heatmap(confusion_matrix, annot=True, cmap='Blues', xticklabels=genres, yticklabels=genres , fmt='g')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
        

# SVM

### best score: 0.5236602052451539

In [ ]:
from sklearn.svm import SVC
# train the model
model = SVC(kernel='rbf' )

model.fit(train_data, train_labels)





# Random Forests

### best score: 0.5653010723123454

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model =RandomForestClassifier(
    n_estimators=300,
    criterion="entropy",
    max_depth=None,
    min_samples_split=2,
)

# train the model
model.fit(train_data, train_labels)

# decision tree

### best score: 0.47594171020071485

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(
    criterion='gini', 
    splitter='best', 
    max_depth=10, 
    min_samples_split=50, 
    min_samples_leaf=10, 
    min_weight_fraction_leaf=0.0, 
    max_features=None, 
    random_state=None, 
    max_leaf_nodes=None, 
    min_impurity_decrease=0.0, 
    class_weight=None, 
    ccp_alpha=0.0
)
model.fit(train_data, train_labels)

In [ ]:
# evaluate the model
confusion_matrix = get_score_confusion_matrix(model, test_Data, test_labels)
draw_confusion_matrix(confusion_matrix)

score = model.score(test_Data, test_labels)

print (f"Total score: {score *100:.2f}%")